# Notes on Jenkins

- Jenkinsfile is a __Pipeline as Code__. It is part of IaC concept.
- Supports both `scripted` (groovy syntax) as well as well as `declarative`(template based) pipeline.
- `Scripted` pipeline starts with __node__ and `declarative` pipeline starts with __pipeline__.

- __when__: For some stage e.g 'test', we only want to run on dev branch or any other branch. For that there is `when` expression.

- environment variable is an importance concept in Jenkins. We can either use the already available tons of Jenkins env var e.g BRANCH_NAME or we can create our own environment variable with `environment {}` expression

- credentials can be used with __1.credentials(for creating credentials in GUI) and 2.credentials binding (for using those credentials in jenkinsfile)__ plugins and then use its function `credentials(id)` to get username and password. This function can be used in our custom environment expression if we need the credentials for multiple stages. If we only need it for one stage, we can use a wrapper `withCredentials()`.

- __tools__: If we want to use some build tools like maven, gradle, jdk etc. for multiple stages, then we can use `tools {}` syntax but the tool name has to be the same as is install in Jenkins (global tool configuration).

- __parameters__: types- string, choice(list), boolean. It's like you define a variable in other programming languages. We can use parameters in `when` expression or any stage as variable.

- `script` block: so that we can modulerize/ make composable jenkinsfile.

- Jenkins need to be installed only on master server and not on agents.

> __jenkins as docker:__ `docker run -p 8080:8080 -p 50000:50000 -v ~/jenkins_home:/var/jenkins_home jenkins/jenkins:lts`

- `# of executors`: number of parallel jobs that it can run.

- __NODE__: Adding a new node means adding a new machine

# Notes on Jenkins Contd.

```groovy
node {
    checkout scm
    /* ... snip ... */
}
```
The `checkout` step will checkout code from source control; `scm` is a special variable which instructs the checkout step to clone the specific revision which triggered this Pipeline run.

### Environment Variable

- An environment directive used in the top-level pipeline block will apply to all steps within the Pipeline.
- An environment directive defined within a stage will only apply the given environment variables to steps within the stage.

### String Interpolation
```groovy
def username = 'Jenkins'
echo 'Hello Mr. ${username}'
echo "I said, Hello Mr. ${username}"
```
Would result in:
```groovy
Hello Mr. ${username}
I said, Hello Mr. Jenkins
```

### Interpolation of sensitive environment variables

> Groovy string interpolation should never be used with credentials.

Groovy string interpolation can leak sensitive environment variables (see: Handling credentials). This is because the sensitive environment variable will be interpolated during Groovy evaluation, and the environment variable’s value could be made available earlier than intended, resulting in sensitive data leaking in various contexts.

```groovy
Jenkinsfile (Declarative Pipeline)
pipeline {
    agent any
    environment {
        EXAMPLE_CREDS = credentials('example-credentials-id')
    }
    stages {
        stage('Example') {
            steps {
                /* WRONG! */
                sh("curl -u ${EXAMPLE_CREDS_USR}:${EXAMPLE_CREDS_PSW} https://example.com/")
                /* CORRECT */
                sh('curl -u $EXAMPLE_CREDS_USR:$EXAMPLE_CREDS_PSW https://example.com/')
                
            }
        }
    }
}
```

Similary, passing special characters `(e.g. / \ $ & % ^ > < | ;)` from a user-defined parameters to shell can result in problems analogous to SQL injection.
```groovy
Jenkinsfile (Declarative Pipeline)
pipeline {
    agent any
    parameters {
        string(name: 'STATEMENT', defaultValue: 'hello; ls /', description: 'What should I say?')
    }
    stages {
        stage('Example') {
            steps {
                /* WRONG! */
                sh("echo ${params.STATEMENT}")
                /* CORRECT */
                sh('echo ${params.STATEMENT}')
            }    
        }
    }
}
```

### Post conditions to handle failure

> `always, unstable, success, failure, changed`

```groovy
Jenkinsfile (Declarative Pipeline)
pipeline {
    agent any
    stages {
        stage('Test') {
            steps {
                sh 'make check'
            }
        }
    }
    post {
        always {
            junit '**/target/*.xml'
        }
        failure {
            mail to: team@example.com, subject: 'The Pipeline failed :('
        }
    }
}
```

---

If you run three shell steps back-to-back, each of those steps has to be started and stopped, requiring connections and resources on the agent and controller to be created and cleaned up. However, if you put all of the commands into a single shell step, then only a single step needs to be started and stopped.

> Instead of creating a series of echo or sh steps, combine them into a single step or script.


# Deployment

- Before deployment we have `BUILD` stage/job that generates the artifacts/executable file let's say jar file which we can deploy to test environment, to artifactory.

- Deployement is finally also done on production environment which is also called `RELEASE`